# Modelo: XGBoost

El objetivo principal de este cuaderno es:
* Entrenar un modelo XGBoost, que es un algoritmo de Gradient Boosting extremadamente eficiente.
* Evaluar su desempeño utilizando el conjunto de validación.
* Obtener métricas finales (accuracy, matriz de confusión, classification report).
* Generar predicciones para Kaggle basadas en este modelo.


### Importaciones


En esta sección cargamos todas las librerías necesarias para entrenamiento, evaluación y generación de predicciones.


In [ ]:
import os, json
import zipfile
import pandas as pd
import numpy as np
from google.colab import userdata

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

try:
    from xgboost import XGBClassifier
    has_xgb = True
except Exception as e:
    print("xgboost no disponible, uso alternativa HistGradientBoosting.")
    has_xgb = False

xgboost no disponible, uso alternativa HistGradientBoosting.


### Traemos los datos desde kaggle

In [ ]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key
assert user and key, "Faltan los secretos KAGGLE_USERNAME/KAGGLE_KEY"
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\MachineLearningPruebas\envs\CursoMachineLearning\Scripts\kaggle.exe\__main__.py", line 4, in <module>
    from kaggle.cli import main
  File "D:\MachineLearningPruebas\envs\CursoMachineLearning\Lib\site-packages\kaggle\__init__.py", line 6, in <module>
    api.authenticate()
    ~~~~~~~~~~~~~~~~^^
  File "D:\MachineLearningPruebas\envs\CursoMachineLearning\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
    ...<3 lines>...
                      self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\elect\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
El sistema no puede encontrar la ruta especificada.
"wc" no se reconoce como un coma

In [3]:
df_train = pd.read_csv("train.csv")
df_test  = pd.read_csv("test.csv")
df_sample = pd.read_csv("submission_example.csv")

df_train.head(3)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264


### Preprocesado de la Data

Para el preprocesado usare la funcion propuesta para la entrega #2 con unas pequeñas adaptaciones

In [4]:
def make_scaler(kind: str):
    kind = (kind or 'standard').lower()
    if kind in ('standard', 'std', 'zscore'):
        return StandardScaler(with_mean=False)  # compatible con matrices dispersas
    else:
        return StandardScaler(with_mean=False)


def build_preprocessor_v2(
    df: pd.DataFrame,
    y_col: str = 'RENDIMIENTO_GLOBAL',
    id_col: str = 'ID',
    num_impute: str = 'median',
    cat_impute: str = 'most_frequent',
    encode_categorical: str = 'onehot',
    ordinal_maps: dict = None,
    scale_numeric: str = 'standard'
):
    data = df.copy()
    y = data.pop(y_col) if y_col in data.columns else None

    # Quitar columnas no predictoras
    for col in [id_col, 'Y_NUMERIC']:
        if col in data.columns:
            data = data.drop(columns=col)

    # Tipos
    num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
    cat_cols = data.select_dtypes(include=['object']).columns.tolist()

    # Ordinal vs OneHot
    ordinal_maps = ordinal_maps or {}
    ordinal_cols = [c for c in cat_cols if c in ordinal_maps]
    onehot_cols  = [c for c in cat_cols if c not in ordinal_maps]

    # Pipelines
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=num_impute)),
        ('scaler', make_scaler(scale_numeric))
    ])

    onehot_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=True))
    ])

    ordinal_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('ordinal', OrdinalEncoder(
            categories=[ordinal_maps[c] for c in ordinal_cols],
            handle_unknown='use_encoded_value', unknown_value=-1
        ))
    ]) if ordinal_cols else 'drop'

    preprocessor = ColumnTransformer(
        transformers=[
            ('num',    num_pipe,    num_cols),
            ('onehot', onehot_pipe, onehot_cols),
            ('ordinal', ordinal_pipe, ordinal_cols)
        ],
        remainder='drop',
        n_jobs=1
    )
    return preprocessor, data, y

In [5]:
ordinal_maps = {
  'E_VALORMATRICULAUNIVERSIDAD': [
      'NO PAGO MATRICULA',
      'MENOS DE 500 MIL',
      'ENTRE 500 MIL Y MENOS DE 1 MILLON',
      'ENTRE 1 MILLON Y MENOS DE 2.5 MILLONES',
      'ENTRE 2.5 MILLONES Y MENOS DE 4 MILLONES',
      'ENTRE 4 MILLONES Y MENOS DE 5.5 MILLONES',
      'ENTRE 5.5 MILLONES Y MENOS DE 7 MILLONES',
      'MAS DE 7 MILLONES'
  ],
  'E_HORASSEMANATRABAJA': [
      '0',
      'MENOS DE 10 HORAS',
      'ENTRE 11 Y 20 HORAS',
      'ENTRE 21 Y 30 HORAS',
      'MAS DE 30 HORAS'
  ],
  'F_ESTRATOVIVIENDA': [
      'SIN ESTRATO', 'ESTRATO 1', 'ESTRATO 2', 'ESTRATO 3', 'ESTRATO 4', 'ESTRATO 5', 'ESTRATO 6'
  ],
  'F_EDUCACIONPADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
    ],
    'F_EDUCACIONMADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
  ]
}

In [6]:
preproc, X_df, y = build_preprocessor_v2(
    df_train,
    y_col='RENDIMIENTO_GLOBAL',
    id_col='ID',
    num_impute='median',
    cat_impute='most_frequent',
    encode_categorical='onehot',
    ordinal_maps=ordinal_maps,
    scale_numeric='standard'
)

In [7]:
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_df, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

### Entrenamiento del modelo XGBoost


Entrenamos un XGBoost con hiperparámetros iniciales.
Este modelo utiliza Gradient Boosting, una técnica que combina muchos árboles débiles para formar un predictor fuerte.
XGBoost es especialmente eficiente gracias a optimizaciones computacionales y regularización integrada.


In [9]:
xgb = Pipeline(steps=[
    ('prep', preproc),
    ('clf', XGBClassifier(
        objective='multi:softmax',
        eval_metric='mlogloss',
        num_class=4,
        n_estimators=900,
        max_depth=None,
        learning_rate=0.005,
        subsample=0.85,
        colsample_bytree=0.8,
        tree_method='hist',
        min_child_weight=5,
        gamma=0.3,
        random_state=42
    ))
])
xgb.fit(X_train, y_train)

NameError: name 'XGBClassifier' is not defined

### Evaluación del modelo


Incluimos:
* accuracy
* matriz de confusión
* classification report

Estas métricas permiten visualizar el comportamiento del modelo y entender sus fortalezas y debilidades.

**Comparación de modelos:** Esperamos que XGBoost ofrezca un balance entre precisión (cercano a Random Forest) y velocidad de entrenamiento (mejor que Random Forest), lo que lo convierte en una opción viable para ciclos iterativos.


In [ ]:
preds_enc = xgb.predict(X_valid)
preds = le.inverse_transform(preds_enc)

y_valid_labels = le.inverse_transform(y_valid)
print("Accuracy:", accuracy_score(y_valid_labels, preds))
print("F1-macro:", f1_score(y_valid_labels, preds, average='macro'))
print("\nReporte:\n", classification_report(y_valid_labels, preds))
print("\nMatriz de confusión:\n", confusion_matrix(y_valid_labels, preds, labels=sorted(le.classes_)))

Accuracy: 0.40664981949458484
F1-macro: 0.3885294407495543

Reporte:
               precision    recall  f1-score   support

        alto       0.48      0.62      0.54     35124
        bajo       0.43      0.54      0.48     34597
  medio-alto       0.31      0.19      0.24     34324
  medio-bajo       0.32      0.27      0.29     34455

    accuracy                           0.41    138500
   macro avg       0.39      0.41      0.39    138500
weighted avg       0.39      0.41      0.39    138500


Matriz de confusión:
 [[21777  3695  5301  4351]
 [ 4543 18583  3856  7615]
 [11648  8139  6632  7905]
 [ 7121 12464  5541  9329]]


### Prediccion sobre el test completo

In [ ]:
X_test = df_test.copy()
for col in ['ID', 'Y_NUMERIC', 'RENDIMIENTO_GLOBAL']:
    if col in X_test.columns:
        X_test = X_test.drop(columns=col)

test_preds_enc = xgb.predict(X_test)

test_preds = le.inverse_transform(test_preds_enc)

submission_xgb = pd.DataFrame({
    'ID': df_test['ID'],
    'RENDIMIENTO_GLOBAL': test_preds
})

submission_xgb.to_csv('submission_xgboost_final.csv', index=False)
submission_xgb.head()

### Mandando a la competencia

In [ ]:
!kaggle competitions submit \
  -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
  -f submission_xgboost_final.csv \
  -m "XGBoost modelo final"